In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.dataloader import DataLoader
from itertools import chain



import sys
import os
current = os.path.dirname(os.path.realpath('.'))
sys.path.append(current)
from TPR_utils import TPR, build_D, DecodedTPR2Tree

from models import *
from data import BinaryT2TDataset

/Users/psoulos/opt/anaconda3/envs/blackboard/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bsz = 1
max_depth = 8
d_filler = 32
d_role = 2**max_depth
from data import BinaryT2TDataset
data_file = '/Users/psoulos/.data/nc_pat/v16/car_cdr_seq/train.xy'
train_data = BinaryT2TDataset(data_file, max_depth=max_depth, device='cpu', filter=None)

In [3]:
class Object(object):
    pass

args = Object()
args.learn_filler_embed = False
args.proj_filler_to_unit_ball = False
tpr = TPR(args=args, num_fillers=len(train_data.ind2vocab), num_roles=2**max_depth-1, d_filler=d_filler, d_role=d_role)
D_l, D_r = build_D(tpr.role_emb)
#E_l, E_r = build_E(tpr.role_emb)

In [4]:
car = lambda x: torch.einsum('bmn,cn->bmc', x.view(bsz, d_filler, d_role), D_l).reshape(bsz, -1)
cdr = lambda x: torch.einsum('bmn,cn->bmc', x.view(bsz, d_filler, d_role), D_r).reshape(bsz, -1)


In [5]:
list_data = list(train_data)
for item in list_data[0:10]:
    t = tpr(item['input'].unsqueeze(0))
    t = t.reshape(bsz,-1)
    #print('Mean {}: '.format(torch.mean(t)))
    #print('Var {}: '.format(torch.var(t)))
    print('Norm {}: '.format(torch.norm(t)))
    num_nodes = torch.sum(list_data[0]['input'] > 0)
    print(num_nodes)
    
    
    

Norm 2.595029830932617: 
tensor(22)
Norm 3.0961191654205322: 
tensor(22)
Norm 2.7655043601989746: 
tensor(22)
Norm 2.778975486755371: 
tensor(22)
Norm 3.0982179641723633: 
tensor(22)
Norm 2.9246740341186523: 
tensor(22)
Norm 2.7730295658111572: 
tensor(22)
Norm 2.628295421600342: 
tensor(22)
Norm 2.927591323852539: 
tensor(22)
Norm 2.5584871768951416: 
tensor(22)


In [21]:
torch.sum(list_data[0]['input'] > 0)

tensor(31)

In [117]:
for i in range(10):
    original_t = tpr(list_data[i]['input'].unsqueeze(0))
    current_t = original_t
    for j in range(4):
        fully_decoded = DecodedTPR2Tree(tpr.unbind(current_t.view(d_filler, d_role).unsqueeze(0), decode=True))
        num_nodes = torch.sum(fully_decoded > 0)
        print(BatchSymbols2NodeTree(fully_decoded, train_data.ind2vocab)[0])
        print('# nodes: {}'.format(num_nodes))
        print('Norm {}: '.format(torch.norm(current_t/torch.sqrt(num_nodes))))
        print('Mean {}: '.format(torch.mean(current_t)))
        print('Var {}: '.format(torch.var(current_t)))

        current_t = car(current_t)

( CDDR ( NP ( DET some ) ( AP ( N teacher ) ) ) ( VP ( AUXPS was ) ( VPPS ( V discovered ) ( PPPS ( PPS by ) ( NP ( DET one ) ( AP ( N firewoman ) ) ) ) ) ) )
# nodes: 22
Norm 0.5532618165016174: 
Mean -0.0003723034169524908: 
Var 0.0008220046875067055: 
( NP ( DET some ) ( AP ( N teacher ) ) )
# nodes: 6
Norm 0.5216130018234253: 
Mean 2.0527826563920826e-05: 
Var 0.00019930145936086774: 
( DET some )
# nodes: 2
Norm 0.4595080018043518: 
Mean -5.1609062211355194e-05: 
Var 5.1553397497627884e-05: 
( some )
# nodes: 1
Norm 0.4504547119140625: 
Mean 9.38193807087373e-07: 
Var 2.4772276447038166e-05: 
( CDDAAR ( NP ( DET some ) ( AP ( ADJ stripped ) ( AP ( ADJ polka-dotted ) ( AP ( ADJ pink ) ( AP ( N cat ) ) ) ) ) ) ( VP ( AUXPS was ) ( VPPS ( V chased ) ( PPPS ( PPS by ) ( NP ( DET some ) ( AP ( N policeman ) ) ) ) ) ) )
# nodes: 31
Norm 0.5560795664787292: 
Mean -0.00014615655527450144: 
Var 0.0011702817864716053: 
( NP ( DET some ) ( AP ( ADJ stripped ) ( AP ( ADJ polka-dotted ) ( AP (

In [24]:
fully_decoded = DecodedTPR2Tree(tpr.unbind(t.view(d_filler, d_role).unsqueeze(0), decode=True))
debug_tree = BatchSymbols2NodeTree(fully_decoded, train_data.ind2vocab)[0]

In [25]:
debug_tree

( CDDR ( NP ( DET some ) ( AP ( N teacher ) ) ) ( VP ( AUXPS was ) ( VPPS ( V discovered ) ( PPPS ( PPS by ) ( NP ( DET one ) ( AP ( N firewoman ) ) ) ) ) ) )

In [30]:
current_t

tensor([[-1.9379e-09, -2.1949e-09, -2.1193e-09,  ...,  3.4313e-09,
          1.7069e-09, -2.9311e-09]])

In [39]:
tpr.filler_emb.weight.shape

torch.Size([106, 32])

In [78]:
dim = 2
for i in range(10):
    dim *= 2
    tensor = torch.empty((dim-1, dim))
    nn.init.orthogonal_(tensor, gain=1)
    print('{}x{}'.format(dim-1,dim))
    print('Mean: {}'.format(tensor.mean()))
    print('Var: {}'.format(tensor.var()))
    #print('Norm: {}'.format(tensor.norm(2,-1)))
    

3x4
Mean: -0.2687870264053345
Var: 0.19391290843486786
7x8
Mean: -0.020638197660446167
Var: 0.1268390417098999
15x16
Mean: 0.013358664698898792
Var: 0.06258230656385422
31x32
Mean: 0.003235600655898452
Var: 0.03127105534076691
63x64
Mean: -0.0019095927709713578
Var: 0.015625229105353355
127x128
Mean: 0.00023950466129463166
Var: 0.007812922820448875
255x256
Mean: -0.0004153121553827077
Var: 0.003906137542799115
511x512
Mean: -0.00011536553938640282
Var: 0.0019531191792339087
1023x1024
Mean: -1.3369688531383872e-05
Var: 0.0009765633149072528
2047x2048
Mean: -4.994465143681737e-06
Var: 0.0004882813664153218
